In [102]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import seaborn as sns
import matplotlib as mpl
from datetime import datetime
import plotly.express as px
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [72]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [73]:
ls'/content/gdrive/My Drive/Thesis'

dataframe_dma_flow_1D.csv     dataframe_flow_variable_1D.csv
dataframe_dma_flow_60min.csv  dataframe_flow_variable_60min.csv
dataframe_flow_1D.csv         DMA_225.csv


In [74]:
#data = pd.read_csv("/content/gdrive/My Drive/Thesis/dataframe_flow_variable_60min.csv")
data = pd.read_csv("/content/gdrive/My Drive/Thesis/DMA_225.csv")

In [75]:
data['Date-time'] = pd.to_datetime(data['Date-time'], dayfirst=True, errors='coerce')

In [76]:
data.rename( columns={'Flow':'DMA_225','Date-time':'Date_time'}, inplace=True )

In [77]:
data_DMA225 = data[['Date_time','DMA_225']]

In [78]:
data_DMA225_indexed = data_DMA225.set_index('Date_time')

In [79]:
data_DMA225_indexed.shape

(35040, 1)

In [80]:
dd = data_DMA225_indexed.resample('D').sum()

In [81]:
dd.shape

(365, 1)

In [82]:
values = dd.values 

In [83]:
scaler = StandardScaler()
scaled = scaler.fit_transform(values)

In [84]:
train_size = int(len(scaled) * 0.70)
test_size = len(scaled) - train_size
train, test = scaled[0:train_size,:], scaled[train_size:len(scaled),:]

In [85]:
print(len(train), len(test))

255 110


In [86]:
# Converting numpy array into matrix:
def matrix_dataset(array_dataset, timesteps=1):
 mat_X, mat_Y = [], []
 for i in range(len(array_dataset)-timesteps-1):
  Z = i+timesteps
  m = array_dataset[i:Z, 0]
  mat_X.append(m)
  n = array_dataset[Z, 0]
  mat_Y.append(n)
 return np.array(mat_X), np.array( mat_Y)

In [89]:
# Preparing the data and reshaping it to input layer (2 dimensions - samples, time steps)
timesteps =  7
train_X,train_y = matrix_dataset(train,timesteps)
test_X, test_y = matrix_dataset(test,timesteps)
print("The train_X and train_y values are:",train_X.shape,train_y.shape)
print("The test_X and test_y values are:",test_X.shape,test_y.shape)



The train_X and train_y values are: (247, 7) (247,)
The test_X and test_y values are: (102, 7) (102,)


In [120]:
model = Sequential()
model.add(Dense(70, input_dim=train_X.shape[1], kernel_initializer='normal', activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(50, kernel_initializer='normal', activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(20, kernel_initializer='normal', activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, kernel_initializer='normal'))

# Compile model
model.compile(loss='mean_squared_error', optimizer='adam')

history = model.fit(x=train_X, y=train_y,validation_data=(test_X,test_y), batch_size=32, epochs=100, verbose=1)


Epoch 1/100
8/8 [==============================] - 1s 22ms/step - loss: 0.7933 - val_loss: 1.1341
Epoch 2/100
8/8 [==============================] - 0s 7ms/step - loss: 0.9522 - val_loss: 1.1483
Epoch 3/100
8/8 [==============================] - 0s 7ms/step - loss: 1.3310 - val_loss: 1.1635
Epoch 4/100
8/8 [==============================] - 0s 8ms/step - loss: 1.1713 - val_loss: 1.1798
Epoch 5/100
8/8 [==============================] - 0s 7ms/step - loss: 0.6802 - val_loss: 1.1911
Epoch 6/100
8/8 [==============================] - 0s 7ms/step - loss: 0.8502 - val_loss: 1.1996
Epoch 7/100
8/8 [==============================] - 0s 8ms/step - loss: 0.5018 - val_loss: 1.1902
Epoch 8/100
8/8 [==============================] - 0s 7ms/step - loss: 0.8000 - val_loss: 1.1562
Epoch 9/100
8/8 [==============================] - 0s 6ms/step - loss: 0.8164 - val_loss: 1.0737
Epoch 10/100
8/8 [==============================] - 0s 6ms/step - loss: 0.5277 - val_loss: 0.9391
Epoch 11/100
8/8 [==========

In [121]:
eval = model.evaluate(x=test_X, y=test_y, verbose=1)
test_pred = model.predict(test_X)

4/4 [==============================] - 0s 2ms/step - loss: 0.6727


In [122]:
#Prediction and getting the accuracy metrics
predict_train=model.predict(train_X)
predict_test=model.predict(test_X)

##Transformback to original form
predict_train_mlp=scaler.inverse_transform(predict_train)
predict_test_mlp=scaler.inverse_transform(predict_test)

In [123]:
### Calculate RMSE performance metrics
import math
from sklearn.metrics import mean_squared_error
trainset = math.sqrt(mean_squared_error(train_y,predict_train_mlp))
testset = math.sqrt(mean_squared_error(test_y,predict_test_mlp))

print("The RMSE prediction value on trainset: ",trainset)
print("The RMSE prediction value on testset: ",testset)

The RMSE prediction value on trainset:  402.1126820842911
The RMSE prediction value on testset:  301.6818557784878


# LSTM Model

In [99]:
# Preparing the data and reshaping it to input layer (2 dimensions - samples, time steps)
timesteps =  7
train_X_lstm,train_y_lstm = matrix_dataset(train,timesteps)
test_X_lstm, test_y_lstm = matrix_dataset(test,timesteps)
print("The train_X and train_y values are:",train_X_lstm.shape,train_y_lstm.shape)
print("The test_X and test_y values are:",test_X_lstm.shape,test_y_lstm.shape)


The train_X and train_y values are: (247, 7) (247,)
The test_X and test_y values are: (102, 7) (102,)


In [100]:
#Reshaping for lstm input
#Feature should be 1 to get lstm input
timesteps = 7
feature = 1
train_X_lstm = train_X_lstm.reshape((train_X_lstm.shape[0], timesteps, feature))
test_X_lstm = test_X_lstm.reshape((test_X_lstm.shape[0], timesteps, feature))
print("The train_X_lstm values after reshaping:",train_X_lstm.shape)
print("The test_X_lstm values after reshaping:",test_X_lstm.shape)

The train_X values after reshaping: (247, 7, 1)
The test_X values after reshaping: (102, 7, 1)


In [281]:
lstm_model=Sequential()
lstm_model.add(LSTM(70,input_shape=(timesteps,feature),activation = 'relu'))
lstm_model.add(Dense(1))
lstm_model.compile(loss='mean_squared_error',optimizer='adam')
lstm_model.fit(train_X_lstm,train_y_lstm,validation_data=(test_X_lstm,test_y_lstm),epochs=125,batch_size=16, 
                    callbacks=[EarlyStopping(monitor='val_loss', patience=10)], verbose=1, shuffle=False)
lstm_model.compile(loss='mean_squared_error',optimizer='adam')

Epoch 1/125
16/16 [==============================] - 1s 21ms/step - loss: 0.5818 - val_loss: 0.8779
Epoch 2/125
16/16 [==============================] - 0s 7ms/step - loss: 0.5127 - val_loss: 0.7355
Epoch 3/125
16/16 [==============================] - 0s 7ms/step - loss: 0.4543 - val_loss: 0.5897
Epoch 4/125
16/16 [==============================] - 0s 7ms/step - loss: 0.3967 - val_loss: 0.4373
Epoch 5/125
16/16 [==============================] - 0s 7ms/step - loss: 0.3593 - val_loss: 0.3328
Epoch 6/125
16/16 [==============================] - 0s 8ms/step - loss: 0.3267 - val_loss: 0.3274
Epoch 7/125
16/16 [==============================] - 0s 7ms/step - loss: 0.2926 - val_loss: 0.3437
Epoch 8/125
16/16 [==============================] - 0s 7ms/step - loss: 0.3160 - val_loss: 0.3330
Epoch 9/125
16/16 [==============================] - 0s 7ms/step - loss: 0.3240 - val_loss: 0.3319
Epoch 10/125
16/16 [==============================] - 0s 7ms/step - loss: 0.3112 - val_loss: 0.4191
Epoch 11

In [282]:
lstm_model.summary()

Model: "sequential_43"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_38 (LSTM)               (None, 70)                20160     
_________________________________________________________________
dense_62 (Dense)             (None, 1)                 71        
Total params: 20,231
Trainable params: 20,231
Non-trainable params: 0
_________________________________________________________________


In [283]:
evaluation = lstm_model.evaluate(x=test_X_lstm, y=test_y_lstm, verbose=1)
test_prediction = lstm_model.predict(test_X_lstm)

4/4 [==============================] - 0s 3ms/step - loss: 0.5736


In [284]:
#Prediction and getting the accuracy metrics
predict_train_lstm=lstm_model.predict(train_X_lstm)
predict_test_lstm=lstm_model.predict(test_X_lstm)

##Transformback to original form
predict_train_lstm=scaler.inverse_transform(predict_train_lstm)
predict_test_lstm=scaler.inverse_transform(predict_test_lstm)

In [290]:
### Calculate RMSE performance metrics
import math
from sklearn.metrics import mean_squared_error
trainset = math.sqrt(mean_squared_error(train_y_lstm,predict_train_lstm))
testset = math.sqrt(mean_squared_error(test_y_lstm,predict_test_lstm))

print("The RMSE prediction value on trainset: ",trainset)
print("The RMSE prediction value on testset: ",testset)

The RMSE prediction value on trainset:  416.08435564836157
The RMSE prediction value on testset:  218.74341239488356


# Stacked LSTM

In [343]:
S_LSTMmodel=Sequential()
S_LSTMmodel.add(LSTM(70,return_sequences=True,input_shape=(timesteps,feature),activation='relu'))
S_LSTMmodel.add(Dropout(0.5))
S_LSTMmodel.add(LSTM(50,return_sequences=True,activation='relu'))
S_LSTMmodel.add(Dropout(0.5))
S_LSTMmodel.add(LSTM(20))
S_LSTMmodel.add(Dropout(0.5))
S_LSTMmodel.add(Dense(1,activation='linear'))
S_LSTMmodel.compile(loss='mean_squared_error',optimizer='adam')
hy= S_LSTMmodel.fit(train_X_lstm,train_y_lstm,validation_data=(test_X_lstm,test_y_lstm),epochs=100,batch_size=16, 
                    callbacks=[EarlyStopping(monitor='val_loss', patience=10)], verbose=1, shuffle=False)
S_LSTMmodel.compile(loss='mean_squared_error',optimizer='adam')

Epoch 1/100
16/16 [==============================] - 4s 55ms/step - loss: 0.5801 - val_loss: 1.1070
Epoch 2/100
16/16 [==============================] - 0s 14ms/step - loss: 0.5360 - val_loss: 1.1591
Epoch 3/100
16/16 [==============================] - 0s 15ms/step - loss: 0.4768 - val_loss: 1.1774
Epoch 4/100
16/16 [==============================] - 0s 14ms/step - loss: 0.4396 - val_loss: 1.0513
Epoch 5/100
16/16 [==============================] - 0s 15ms/step - loss: 0.3616 - val_loss: 0.6155
Epoch 6/100
16/16 [==============================] - 0s 16ms/step - loss: 0.3645 - val_loss: 0.4162
Epoch 7/100
16/16 [==============================] - 0s 15ms/step - loss: 0.3194 - val_loss: 0.5446
Epoch 8/100
16/16 [==============================] - 0s 15ms/step - loss: 0.4146 - val_loss: 0.4919
Epoch 9/100
16/16 [==============================] - 0s 15ms/step - loss: 0.3405 - val_loss: 0.4808
Epoch 10/100
16/16 [==============================] - 0s 14ms/step - loss: 0.3247 - val_loss: 0.4989

In [344]:
S_LSTMmodel.summary()

Model: "sequential_55"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_72 (LSTM)               (None, 7, 70)             20160     
_________________________________________________________________
dropout_50 (Dropout)         (None, 7, 70)             0         
_________________________________________________________________
lstm_73 (LSTM)               (None, 7, 50)             24200     
_________________________________________________________________
dropout_51 (Dropout)         (None, 7, 50)             0         
_________________________________________________________________
lstm_74 (LSTM)               (None, 20)                5680      
_________________________________________________________________
dropout_52 (Dropout)         (None, 20)                0         
_________________________________________________________________
dense_74 (Dense)             (None, 1)               

In [345]:
evaluation_SL = lstm_model.evaluate(x=test_X_lstm, y=test_y_lstm, verbose=1)
test_prediction_SL = lstm_model.predict(test_X_lstm)

4/4 [==============================] - 0s 3ms/step - loss: 0.4143


In [346]:
#Prediction and getting the accuracy metrics
predict_train_Slstm=S_LSTMmodel.predict(train_X_lstm)
predict_test_Slstm=S_LSTMmodel.predict(test_X_lstm)

##Transformback to original form
predict_train_Slstm=scaler.inverse_transform(predict_train_Slstm)
predict_test_Slstm=scaler.inverse_transform(predict_test_Slstm)

In [347]:
### Calculate RMSE performance metrics
import math
from sklearn.metrics import mean_squared_error
trainset_Slstm = math.sqrt(mean_squared_error(train_y_lstm,predict_train_Slstm))
testset_Slstm = math.sqrt(mean_squared_error(test_y_lstm,predict_test_Slstm))

print("The S_lstm RMSE prediction value on trainset: ",trainset_Slstm)
print("The S_lstm RMSE prediction value on testset: ",testset_Slstm)

The S_lstm RMSE prediction value on trainset:  416.5407588753957
The S_lstm RMSE prediction value on testset:  288.0621748681771
